# Tmp - Corrección de códigos de rutas de encuestas Guadalajara

In [1]:
#Autopep8

## Import libraries

In [2]:
first_folder_path = '../'

In [3]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import src
else:
    import src

## Load data

In [4]:
surveys_file = 'Encuesta_caminata(Guadalajara)(1-553).csv'
surveys = pd.read_csv(first_folder_path + f"data/output/table/encuestas/{surveys_file}")

# Show
print(surveys.shape)
surveys.head(2)

(553, 65)


,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
0,1,6/28/24 12:24:50,6/28/24 12:25:46,anonymous,NaN,NaN,DS,6/25/2024,001,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
1,2,6/28/24 12:25:58,6/28/24 12:28:18,anonymous,NaN,NaN,DS,6/25/2024,002,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [5]:
surveys.Encuestador.unique()

array(['DS', 'JM', 'DP', 'DP ', '049', 'MV', 'RV', 'EE1V002', 'EE1V004',
       'EE1V001', 'EE1V005', 'EE1V003', 'Jesus Corona', 'Jesús Corona',
       'rv', 'MeV'], dtype=object)

## Missing poll data

In [110]:
# Looking for code DP1V008's data
encuestador_lst = ['DP', 'DP ']

surveys_look = surveys.loc[(surveys.Encuestador.isin(encuestador_lst))&(surveys["Territorio de análisis"]=='TA_Providencia')&(surveys["Encuesta #"]=='008')].copy()

surveys_look[['ID', # Field 1
              'Hora de inicio', # Field 2
              'Hora de finalización', # Field 3
              'Correo electrónico', # Field 4
              'Nombre', # Field 5
              'Hora de la última modificación', # Field 6 
              'Encuestador', # Field 7
              'Fecha de realización', # Field 8 
              'Encuesta #', # Field 9
              'Territorio de análisis', # Field 10
              'Lugar donde se realizó la encuesta', # Field 13
              'Género', # Field 14
              'Edad (años)', # Field 15
              'Estrato socioeconómico (NO APLICA)', # Field 16
              'Ocupación principal', # Field 17
              'Nivel educativo completado', # Field 18
              '¿Poseen alguno de los siguientes vehículos en su vivienda?', # Field 19
              'Indique su lugar de residencia (Municipio/comuna/barrio)', # Field 20
              '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?', # Field 21
              '¿Cómo suple las necesidades de equipamientos educativos?']] # Field 22 y Field 23

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,Lugar donde se realizó la encuesta,Género,Edad (años),Estrato socioeconómico (NO APLICA),Ocupación principal,Nivel educativo completado,¿Poseen alguno de los siguientes vehículos en su vivienda?,Indique su lugar de residencia (Municipio/comuna/barrio),¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?,¿Cómo suple las necesidades de equipamientos educativos?
140,141,7/15/24 14:47:07,7/15/24 14:53:13,anonymous,NaN,NaN,DP,6/28/2024,008,TA_Providencia,Parque Rubén Darío,Masculino,50-59,NaN,Autoempleo,Pregrado,Auto - coche;,Providencia,Si,NaN


In [103]:
surveys_look[['¿Cuáles?', # Field 24
              '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?', # Field 25
              '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)', # Field 26
              'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?', # Field 27
              'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)', # Field 28
              'Debido a la lejanía del equipamiento X', # Field 29
              'Debido a la ausencia de andenes', # Field 30
              'Porque no me siento seguro/a frente a otros vehículos', # Field 31
              'Porque no me siento seguro/a frente a posibles delitos en mi trayecto', # Field 32
              'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles', # Field 33
              'Debido a que no me siento cómodo/a por el calor', # Field 34
              'Debido a que siento que los andenes muy saturados de gente', # Field 35
              'Debido a que el paisaje a lo largo del recorrido no es agradable', # Field 36
              'La cercanía del equipamiento X',  # Field 37
              'La presencia de andenes', # Field 38
              'Sentirme seguro/a frente a otros vehículos', # Field 39
              'Sentirme seguro/a frente a posibles delitos en mi trayecto', # Field 40
              'La posibilidad de hacer múltiples vueltas/trámites en mi recorrido', # Field 41
              'La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido', # Field 42
              'Sentirme cómodo/a por la sombra generada por la vegetación']] # Field 43

,¿Cuáles?,¿Hacia qué tipo de equipamiento/sitio de interés se dirige?,¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado),"En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?","En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)",Debido a la lejanía del equipamiento X,Debido a la ausencia de andenes,Porque no me siento seguro/a frente a otros vehículos,Porque no me siento seguro/a frente a posibles delitos en mi trayecto,Debido al esfuerzo que debo hacer a causa de la pendiente de las calles,Debido a que no me siento cómodo/a por el calor,Debido a que siento que los andenes muy saturados de gente,Debido a que el paisaje a lo largo del recorrido no es agradable,La cercanía del equipamiento X,La presencia de andenes,Sentirme seguro/a frente a otros vehículos,Sentirme seguro/a frente a posibles delitos en mi trayecto,La posibilidad de hacer múltiples vueltas/trámites en mi recorrido,La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido,Sentirme cómodo/a por la sombra generada por la vegetación
140,Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Muy importante,Muy importante,Importante,No es importante,Muy importante


In [111]:
surveys_look[['Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido', # Field 44
              'Que el paisaje sea agradable', # Field 45
              '¿Se encuentra usted en alguna condición de discapacidad?', # Field 46
              'Indique cuál de las siguientes condiciones de discapacidad presenta', # Field 47
              'Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)', # Field 48
              'Indique la hora de salida de su lugar de origen de recorrido (formato militar)', # Field 49
              'Ingrese los minutos de salida', # Field 50
              #'Indique el lugar de destino de su recorrido lo más precios posible (Punto de referencia, intersección más cercana)', # Field 51
              '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)', # Field 52
              'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...', # Field 53
              '¿Por qué razón utiliza la CAMINATA como medio de transporte?', # Field 54
              '¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?']] # Field 55

,Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido,Que el paisaje sea agradable,¿Se encuentra usted en alguna condición de discapacidad?,Indique cuál de las siguientes condiciones de discapacidad presenta,"Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)",Indique la hora de salida de su lugar de origen de recorrido (formato militar),Ingrese los minutos de salida,¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos),Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...,¿Por qué razón utiliza la CAMINATA como medio de transporte?,¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?
140,Muy importante,Muy importante,No,NaN,Calle Álcamo,400.0,0.0,8.0,"Calle Álcamo cerca del Banregio, siguiendo por...",Paseo de su mascota;,Carro particular - coche


In [112]:
surveys_look[['Column', # Field 56
              '2', # Field 57
              '3', # Field 58
              '4', # Field 59
              '5', # Field 60
              '6', # Field 61
              '7', # Field 62
              '8', # Field 63
              '9', # Field 64
              '10', # Field 65
              '11', # Field 66
              '12']] # Field 67

,Column,2,3,4,5,6,7,8,9,10,11,12
140,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


## Filter and fix codes

In [6]:
# Pollster a) - Jesus
encuestador_lst = ['Jesus Corona', 'Jesús Corona']
surveys_jesus = surveys.loc[surveys.Encuestador.isin(encuestador_lst)].copy()

# Rename codes
surveys_jesus["code"] = 0
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 V-001', "code"] = 'JC1V001'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 V-002', "code"] = 'JC1V002'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 V-003', "code"] = 'JC1V003'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 V-004', "code"] = 'JC1V004'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 V-005', "code"] = 'JC1V005'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 V-006', "code"] = 'JC1V006'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-01 M-007', "code"] = 'JC1V007'
surveys_jesus.loc[surveys_jesus["Encuesta #"]=='JC-1 M-008', "code"] = 'JC1V008'
surveys_jesus

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,4,5,6,7,8,9,10,11,12,code
528,529,5/9/25 11:56:46,5/9/25 12:27:04,anonymous,NaN,NaN,Jesus Corona,3/18/2025,JC-1 V-001,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V001
529,530,5/9/25 12:52:39,5/9/25 12:53:11,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",JC1V002
530,531,5/9/25 12:53:14,5/9/25 12:57:26,anonymous,NaN,NaN,Jesús Corona,3/18/2025,JC-01 V-003,TA_Providencia,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,En desacuerdo,JC1V003
531,532,5/9/25 12:57:29,5/9/25 14:43:21,anonymous,NaN,NaN,Jesús Corona,5/24/2025,JC-1 V-004,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V004
532,533,5/9/25 14:43:23,5/9/25 14:50:40,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-1 V-005,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,De acuerdo,De acuerdo,JC1V005
533,534,5/9/25 14:50:42,5/9/25 14:56:06,anonymous,NaN,NaN,Jesús Corona,4/24/2025,JC-01 V-006,TA_Providencia,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo",JC1V006
534,535,5/9/25 14:56:21,5/9/25 15:08:22,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-01 M-007,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,JC1V007
535,536,5/9/25 15:08:24,5/9/25 15:20:28,anonymous,NaN,NaN,Jesús Corona,5/8/2025,JC-1 M-008,TA_Providencia,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,JC1V008


In [7]:
# Pollster b) - Meli
encuestador_lst = ['MeV']
surveys_meli = surveys.loc[surveys.Encuestador.isin(encuestador_lst)].copy()

# Rename codes
surveys_meli["code"] = 0
surveys_meli.loc[surveys_meli["Encuesta #"]=='0001', "code"] = 'MeV1V001'
surveys_meli.loc[surveys_meli["Encuesta #"]=='002', "code"] = 'MeV1V002'
surveys_meli.loc[surveys_meli["Encuesta #"]=='003', "code"] = 'MeV1V003'
surveys_meli.loc[surveys_meli["Encuesta #"]=='004', "code"] = 'MeV1V004'
surveys_meli.loc[surveys_meli["Encuesta #"]=='005', "code"] = 'MeV1V005'
surveys_meli.loc[surveys_meli["Encuesta #"]=='006', "code"] = 'MeV1V006'
surveys_meli.loc[surveys_meli["Encuesta #"]=='007', "code"] = 'MeV1V007'
surveys_meli.loc[surveys_meli["Encuesta #"]=='008', "code"] = 'MeV1V008'
surveys_meli.loc[surveys_meli["Encuesta #"]=='009', "code"] = 'MeV1V009'
surveys_meli.loc[surveys_meli["Encuesta #"]=='010', "code"] = 'MeV1V010'
surveys_meli

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,4,5,6,7,8,9,10,11,12,code
543,544,5/11/25 19:50:28,5/11/25 19:59:26,anonymous,NaN,NaN,MeV,3/18/2025,0001,TA_Providencia,...,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V001
544,545,5/11/25 20:00:52,5/11/25 20:07:21,anonymous,NaN,NaN,MeV,3/18/2025,002,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V002
545,546,5/11/25 20:07:30,5/11/25 20:15:38,anonymous,NaN,NaN,MeV,3/18/2025,003,TA_Providencia,...,Totalmente en desacuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V003
546,547,5/11/25 20:15:47,5/11/25 20:17:53,anonymous,NaN,NaN,MeV,3/18/2025,004,TA_Providencia,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",MeV1V004
547,548,5/11/25 20:18:06,5/11/25 20:29:49,anonymous,NaN,NaN,MeV,3/18/2025,005,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,MeV1V005
548,549,5/11/25 20:29:58,5/11/25 20:32:13,anonymous,NaN,NaN,MeV,3/18/2025,006,TA_Providencia,...,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V006
549,550,5/11/25 20:32:22,5/11/25 20:36:13,anonymous,NaN,NaN,MeV,3/18/2025,007,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,En desacuerdo,MeV1V007
550,551,5/11/25 20:36:23,5/11/25 20:41:02,anonymous,NaN,NaN,MeV,3/18/2025,008,TA_Providencia,...,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,MeV1V008
551,552,5/11/25 20:41:04,5/11/25 20:45:47,anonymous,NaN,NaN,MeV,3/18/2025,009,TA_Providencia,...,Totalmente de acuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,MeV1V009
552,553,5/11/25 20:45:49,5/11/25 20:49:13,anonymous,NaN,NaN,MeV,3/18/2025,010,TA_Providencia,...,De acuerdo,En desacuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,Totalmente en desacuerdo,En desacuerdo,Totalmente en desacuerdo,MeV1V010


In [32]:
# Pollster b) - Ross
encuestador_lst = ['RV', 'rv']
surveys_ross = surveys.loc[(surveys.Encuestador.isin(encuestador_lst))&(surveys["Territorio de análisis"]=='TA_Providencia')].copy()

a="""
# Rename codes
surveys_ross["code"] = 0
surveys_ross.loc[surveys_ross["Encuesta #"]=='0001', "code"] = 'MeV1V001'
surveys_ross.loc[surveys_ross["Encuesta #"]=='002', "code"] = 'MeV1V002'
surveys_ross.loc[surveys_ross["Encuesta #"]=='003', "code"] = 'MeV1V003'
surveys_ross.loc[surveys_ross["Encuesta #"]=='004', "code"] = 'MeV1V004'
surveys_ross.loc[surveys_ross["Encuesta #"]=='005', "code"] = 'MeV1V005'
surveys_ross.loc[surveys_ross["Encuesta #"]=='006', "code"] = 'MeV1V006'
surveys_ross.loc[surveys_ross["Encuesta #"]=='007', "code"] = 'MeV1V007'
surveys_ross.loc[surveys_ross["Encuesta #"]=='008', "code"] = 'MeV1V008'
surveys_ross.loc[surveys_ross["Encuesta #"]=='009', "code"] = 'MeV1V009'
surveys_ross.loc[surveys_ross["Encuesta #"]=='010', "code"] = 'MeV1V010'
surveys_ross"""

In [25]:
surveys.columns

Index(['ID', 'Hora de inicio', 'Hora de finalización', 'Correo electrónico',
       'Nombre', 'Hora de la última modificación', 'Encuestador',
       'Fecha de realización', 'Encuesta #', 'Territorio de análisis',
       'Lugar donde se realizó la encuesta', 'Género', 'Edad (años)',
       'Estrato socioeconómico (NO APLICA)', 'Ocupación principal',
       'Nivel educativo completado',
       '¿Poseen alguno de los siguientes vehículos en su vivienda?',
       'Indique su lugar de residencia (Municipio/comuna/barrio)',
       '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?',
       '¿Cómo suple las necesidades de equipamientos educativos?',
       '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?',
       '¿Cuáles?',
       '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?',
       '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
       'En una semana típica, ¿Cuántas veces 

In [33]:
surveys_ross[['ID', 	
              'Hora de inicio',
              'Hora de finalización',
              'Correo electrónico',
              'Nombre',
              'Hora de la última modificación',
              'Encuestador',
              'Fecha de realización',
              'Encuesta #',
              'Territorio de análisis',
              'Lugar donde se realizó la encuesta', 
              'Género', 
              'Edad (años)',
              'Estrato socioeconómico (NO APLICA)', 
              'Ocupación principal',
              'Nivel educativo completado',
              '¿Poseen alguno de los siguientes vehículos en su vivienda?',
              'Indique su lugar de residencia (Municipio/comuna/barrio)',
              ]]

,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,Lugar donde se realizó la encuesta,Género,Edad (años),Estrato socioeconómico (NO APLICA),Ocupación principal,Nivel educativo completado,¿Poseen alguno de los siguientes vehículos en su vivienda?,Indique su lugar de residencia (Municipio/comuna/barrio)
510,511,5/7/25 23:11:14,5/7/25 23:16:03,anonymous,NaN,NaN,RV,3/18/2025,RV1V001,TA_Providencia,Parque Rubén Darío,Femenino,50-59,NaN,Independiente,Pregrado,Auto - coche;,"Tlajomulco, Real del valle"
511,512,5/7/25 23:16:10,5/7/25 23:18:09,anonymous,NaN,NaN,RV,3/18/2025,RV1V002,TA_Providencia,Parque Rubén Darío,Masculino,20-29,NaN,Independiente,Técnico,Auto - coche;Motocicleta;Bicicleta;,"Zapopan, Loma Bonita"
512,513,5/7/25 23:18:11,5/7/25 23:20:00,anonymous,NaN,NaN,RV,3/18/2025,RV1V003,TA_Providencia,Parque Rubén Darío,Masculino,40-49,NaN,Independiente,Técnico,Auto - coche;Motocicleta;Bicicleta;,"Tlaquepaque, Buenos Aires"
513,514,5/7/25 23:20:05,5/7/25 23:26:07,anonymous,NaN,NaN,RV,3/18/2025,RV1V004,TA_Providencia,Parque Rubén Darío,Masculino,50-59,NaN,Independiente,Pregrado,Auto - coche;Bicicleta;,"Zapopan, Puerta del Bosque"
514,515,5/7/25 23:26:15,5/7/25 23:30:25,anonymous,NaN,NaN,RV,3/18/2025,RV1V005,TA_Providencia,Parque Rubén Darío,Masculino,30-39,NaN,Empleado/a,Pregrado,Auto - coche;,"Guadalajara, Providencia"
515,516,5/7/25 23:30:37,5/7/25 23:32:11,anonymous,NaN,NaN,RV,3/18/2025,RV1V006,TA_Providencia,Parque Rubén Darío,Femenino,50-59,NaN,Empleado/a,Secundaria,Ninguno;,"Guadalajara, Hidalgo"
516,517,5/7/25 23:32:20,5/7/25 23:36:39,anonymous,NaN,NaN,RV,3/18/2025,RVV1V007,TA_Providencia,Parque Rubén Darío,Masculino,50-59,NaN,Independiente,Técnico,Auto - coche;Bicicleta;,"Guadalajara, Providencia"
517,518,5/7/25 23:36:41,5/7/25 23:42:13,anonymous,NaN,NaN,RV,3/18/2025,RV1V008,TA_Providencia,Parque Rubén Darío,Masculino,10-19,NaN,Empleado/a,Técnico,Bicicleta;,"Zapopan, Arenales Tapatíos"
518,519,5/7/25 23:42:16,5/7/25 23:46:28,anonymous,NaN,NaN,RV,3/18/2025,RV1V009,TA_Providencia,Parque Rubén Darío,Masculino,30-39,NaN,Empleado/a,Pregrado,Auto - coche;,"Guadalajara, Providencia"
519,520,5/7/25 23:46:32,5/7/25 23:50:05,anonymous,NaN,NaN,RV,3/18/2025,RV1V010,TA_Providencia,Parque Rubén Darío,Masculino,40-49,NaN,Empleado/a,Pregrado,Auto - coche;,"Guadalajara, Providencia"


In [34]:
surveys_ross[['ID','Encuesta #',
              '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?',
              '¿Cómo suple las necesidades de equipamientos educativos?',
              '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?',
              '¿Cuáles?',
              '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?',
              '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
              'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?',
              'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)']]

,ID,Encuesta #,¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?,¿Cómo suple las necesidades de equipamientos educativos?,¿Cómo suple las necesidades de equipamientos de cultura o deportivos?,¿Cuáles?,¿Hacia qué tipo de equipamiento/sitio de interés se dirige?,¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado),"En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?","En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)"
510,511,RV1V001,Si,NaN,NaN,Parques;,Cultural/Deportivo,Carro particular - coche,1,NaN
511,512,RV1V002,Si,NaN,NaN,Parques;,Cultural/Deportivo,Motocicleta,5 o más,NaN
512,513,RV1V003,Si,NaN,NaN,Parques;,Cultural/Deportivo,Carro particular - coche,4,NaN
513,514,RV1V004,Si,NaN,NaN,Parques;,Cultural/Deportivo,Caminata,NaN,5 o más
514,515,RV1V005,Si,NaN,NaN,Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más
515,516,RV1V006,Si,NaN,NaN,Comercial;Parques;,Cultural/Deportivo,Bus,3,NaN
516,517,RVV1V007,Si,NaN,NaN,Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más
517,518,RV1V008,Si,NaN,NaN,Parques;,Cultural/Deportivo,Caminata,NaN,1
518,519,RV1V009,Si,NaN,NaN,Comercial;Parques;,Cultural/Deportivo,Caminata,NaN,5 o más
519,520,RV1V010,Si,NaN,NaN,Salud;Parques;Comercial;,Cultural/Deportivo,Caminata,NaN,5 o más


In [35]:
surveys_ross[['Debido a la lejanía del equipamiento X',
       'Debido a la ausencia de andenes',
       'Porque no me siento seguro/a frente a otros vehículos',
       'Porque no me siento seguro/a frente a posibles delitos en mi trayecto',
       'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles',
       'Debido a que no me siento cómodo/a por el calor',
       'Debido a que siento que los andenes muy saturados de gente',
       'Debido a que el paisaje a lo largo del recorrido no es agradable',
       'La cercanía del equipamiento X', 'La presencia de andenes',
       'Sentirme seguro/a frente a otros vehículos',
       'Sentirme seguro/a frente a posibles delitos en mi trayecto',
       'La posibilidad de hacer múltiples vueltas/trámites en mi recorrido',
       'La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido',
       'Sentirme cómodo/a por la sombra generada por la vegetación',
       'Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido',
       'Que el paisaje sea agradable',
       '¿Se encuentra usted en alguna condición de discapacidad?',
       'Indique cuál de las siguientes condiciones de discapacidad presenta']]

,Debido a la lejanía del equipamiento X,Debido a la ausencia de andenes,Porque no me siento seguro/a frente a otros vehículos,Porque no me siento seguro/a frente a posibles delitos en mi trayecto,Debido al esfuerzo que debo hacer a causa de la pendiente de las calles,Debido a que no me siento cómodo/a por el calor,Debido a que siento que los andenes muy saturados de gente,Debido a que el paisaje a lo largo del recorrido no es agradable,La cercanía del equipamiento X,La presencia de andenes,Sentirme seguro/a frente a otros vehículos,Sentirme seguro/a frente a posibles delitos en mi trayecto,La posibilidad de hacer múltiples vueltas/trámites en mi recorrido,La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido,Sentirme cómodo/a por la sombra generada por la vegetación,Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido,Que el paisaje sea agradable,¿Se encuentra usted en alguna condición de discapacidad?,Indique cuál de las siguientes condiciones de discapacidad presenta
510,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,En desacuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,Muy importante,No,NaN
514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Importante,Importante,Muy importante,No es importante,No es importante,Importante,Importante,Importante,No,NaN
515,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,En desacuerdo,Totalmente en desacuerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Muy importante,Muy importante,Importante,Es poco importante,Importante,Moderadamente importante,Moderadamente importante,No,NaN
517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Importante,Muy importante,Muy importante,Muy importante,Muy importante,Es poco importante,Muy importante,No es importante,Muy importante,No,NaN
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Importante,Muy importante,Moderadamente importante,Importante,Moderadamente importante,No es importante,Es poco importante,No,NaN
519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muy importante,Muy importante,Importante,Importante,Importante,Moderadamente importante,Importante,Muy importante,Muy importante,No,NaN


In [60]:
surveys_ross[['Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)',
              'Indique la hora de salida de su lugar de origen de recorrido (formato militar)',
              'Ingrese los minutos de salida',
              #'Indique el lugar de destino de su recorrido lo más precios posible (Punto de referencia, intersección más cercana)',
              '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)',
              'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...',
              '¿Por qué razón utiliza la CAMINATA como medio de transporte?',
              '¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?'
             ]]

,"Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)",Indique la hora de salida de su lugar de origen de recorrido (formato militar),Ingrese los minutos de salida,¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos),Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...,¿Por qué razón utiliza la CAMINATA como medio de transporte?,¿Qué modo de transporte alternativo utiliza cuando por alguna razón no puede utilizar la caminata como modo de transporte?
510,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,Eulogio Parra y Rubén Darío,17.0,30.0,12.0,Inicia en Eulogio Parra entre Ontario y Rubén ...,Pasear mascota;,Carro particular - coche
514,Otawa esquina con Florencia,17.0,0.0,5.0,"Inicia en otawa esquina con Florencia, camina ...",Por cercanía al lugar de destino;,Carro particular - coche
515,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,Calle Florencia esquina con Monza,18.0,10.0,5.0,"Inicia en Monza con Florencia, camina sobre Fl...",Pasear mascota;,Carro particular - coche
517,Av. Terranova,18.0,35.0,20.0,"Inicia en Av Terranova y Pablo Neruda, camina ...",Distracción;,Bus
518,Terranova esquina con José María VIgil,18.0,40.0,5.0,Camina por Terranova y José María Vigil hacia ...,Pasear mascota;,Carro particular - coche
519,"José María Vigil, entre Terranova y Rubén Darío",18.0,15.0,5.0,Inicia sobre José María Vigil casi esquina con...,Pasear mascota;,Carro particular - coche


In [50]:
surveys_ross[['ID','Encuesta #','Column', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]

,ID,Encuesta #,Column,2,3,4,5,6,7,8,9,10,11,12
510,511,RV1V001,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
511,512,RV1V002,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
512,513,RV1V003,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
513,514,RV1V004,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo
514,515,RV1V005,De acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo
515,516,RV1V006,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
516,517,RVV1V007,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,De acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,En desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo
517,518,RV1V008,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,De acuerdo
518,519,RV1V009,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente en desacuerdo,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo
519,520,RV1V010,Totalmente de acuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,En desacuerdo,De acuerdo
